In [1]:
import torch
import torch.nn as nn
import numpy as np
torch.__version__

'1.9.1+cpu'

# logistic回归实战
在这一章里面，我们将处理一下结构化数据，并使用logistic回归对结构化数据进行简单的分类。
## logistic回归介绍
logistic回归是一种广义线性回归（generalized linear model），与多重线性回归分析有很多相同之处。它们的模型形式基本上相同，都具有 wx + b，其中w和b是待求参数，其区别在于他们的因变量不同，多重线性回归直接将wx+b作为因变量，即y =wx+b,而logistic回归则通过函数L将wx+b对应一个隐状态p，p =L(wx+b),然后根据p 与1-p的大小决定因变量的值。如果L是logistic函数，就是logistic回归，如果L是多项式函数就是多项式回归。

说的更通俗一点，就是logistic回归会在线性回归后再加一层logistic函数的调用。

logistic回归主要是进行二分类预测，我们在激活函数时候讲到过 Sigmod函数，Sigmod函数是最常见的logistic函数，因为Sigmod函数的输出的是是对于0~1之间的概率值，当概率大于0.5预测为1，小于0.5预测为0。

下面我们就来使用公开的数据来进行介绍

## UCI German Credit  数据集

UCI German Credit是UCI的德国信用数据集，里面有原数据和数值化后的数据。

German Credit数据是根据个人的银行贷款信息和申请客户贷款逾期发生情况来预测贷款违约倾向的数据集，数据集包含24个维度的，1000条数据，

在这里我们直接使用处理好的数值化的数据，作为展示。

[地址](https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/)

## 代码实战
我们这里使用的 german.data-numeric是numpy处理好数值化数据，我们直接使用numpy的load方法读取即可

In [2]:
data=np.loadtxt("german.data-numeric")

数据读取完成后我们要对数据做一下归一化的处理

In [3]:
n,l=data.shape
for j in range(l-1):
    meanVal=np.mean(data[:,j])
    stdVal=np.std(data[:,j])
    data[:,j]=(data[:,j]-meanVal)/stdVal

打乱数据

In [7]:
np.random.shuffle(data)

区分训练集和测试集，由于这里没有验证集，所以我们直接使用测试集的准确度作为评判好坏的标准

区分规则：900条用于训练，100条作为测试

german.data-numeric的格式为，前24列为24个维度，最后一个为要打的标签（0，1），所以我们将数据和标签一起区分出来

In [8]:
train_cnt = 900
train_data=data[:train_cnt,:l-1]
train_lab=data[:train_cnt,l-1]-1
test_data=data[train_cnt:,:l-1]
test_lab=data[train_cnt:,l-1]-1

【TODO】定义模型，训练，测试

### 使用torch库

In [9]:
# 数据处理
from torch.autograd import Variable
_train_data = Variable(torch.from_numpy(train_data).float()) # 转换成可以反向传播的张量Variable
_train_lab = Variable(torch.from_numpy(train_lab).long())
_test_data = Variable(torch.from_numpy(test_data).float())
_test_lab = Variable(torch.from_numpy(test_lab).long())

# 模型定义
class LR(nn.Module):
    def __init__(self, input_size):
        super(LR, self).__init__()
        self.func = nn.Linear(input_size, 2) #两个参数分别表示一条数据的输入特征数和分类器的类数
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        dataout = self.sigmoid(self.func(x))
        return dataout
    
# 训练

model = LR(train_data.shape[1])
lossFunc = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

epochs = 4000
for epoch in range(epochs):
    outs = model(_train_data)
    loss = lossFunc(outs, _train_lab)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss:{:.4f}'.format(epoch, epochs, loss.item()))
        
# 测试
testouts = model(_test_data)
# print(testouts[:5])
_, predicts = torch.max(testouts.data, 1)
# print(predicts[:5])
correct = (predicts == _test_lab).sum()
print('----------------------')
print('test accuracy: ', correct / test_data.shape[0])
    

Epoch [0/4000], Loss:0.6998
Epoch [100/4000], Loss:0.6864
Epoch [200/4000], Loss:0.6740
Epoch [300/4000], Loss:0.6628
Epoch [400/4000], Loss:0.6528
Epoch [500/4000], Loss:0.6437
Epoch [600/4000], Loss:0.6357
Epoch [700/4000], Loss:0.6285
Epoch [800/4000], Loss:0.6221
Epoch [900/4000], Loss:0.6164
Epoch [1000/4000], Loss:0.6113
Epoch [1100/4000], Loss:0.6067
Epoch [1200/4000], Loss:0.6025
Epoch [1300/4000], Loss:0.5987
Epoch [1400/4000], Loss:0.5953
Epoch [1500/4000], Loss:0.5922
Epoch [1600/4000], Loss:0.5893
Epoch [1700/4000], Loss:0.5866
Epoch [1800/4000], Loss:0.5842
Epoch [1900/4000], Loss:0.5820
Epoch [2000/4000], Loss:0.5799
Epoch [2100/4000], Loss:0.5779
Epoch [2200/4000], Loss:0.5761
Epoch [2300/4000], Loss:0.5744
Epoch [2400/4000], Loss:0.5728
Epoch [2500/4000], Loss:0.5714
Epoch [2600/4000], Loss:0.5700
Epoch [2700/4000], Loss:0.5686
Epoch [2800/4000], Loss:0.5674
Epoch [2900/4000], Loss:0.5662
Epoch [3000/4000], Loss:0.5651
Epoch [3100/4000], Loss:0.5641
Epoch [3200/4000], L

### 不使用torch库（自行编写代码）

In [18]:
# 简单逻辑回归
def sigmoid(num):
    return 1.0 / (1 + np.exp(-num))

def grad_ascent(train_data, train_lab):
    m, n =  np.shape(train_data)
    w = np.ones(n)
    alpha = 0.01
    max_cycle = 200
    for i in range(max_cycle):
        s = sigmoid(np.dot(train_data, w))
        error = train_lab - s
        w += alpha * np.dot(train_data.transpose(), error)
    return w

w = grad_ascent(train_data, train_lab)
predict_lab = np.round(sigmoid(np.dot(test_data, w)))
fault_cnt = 0
for i in range(np.shape(test_data)[0]):
    if not predict_lab[i] == test_lab[i]:
        fault_cnt += 1
print("accuracy: ", 1 - fault_cnt / np.shape(test_data)[0])


accuracy:  0.71


In [15]:
# 逻辑回归 + 随机梯度下降
def sigmoid(num):
    return 1.0 / (1 + np.exp(-num))

def grad_ascent(train_data, train_lab):
    m, n =  np.shape(train_data)
    w = np.ones(n)
    # alpha = 0.01
    max_cycle = 200
    for i in range(max_cycle):
        data_index = list(range(m))
        for j in range(m // 2):
            alpha = 1.0 / (1 + i + j) + 0.001
            rand_index = int(np.random.uniform(0, len(data_index)))
            data = train_data[data_index[rand_index]]
            s = sigmoid(np.dot(data, w))
            error = train_lab[data_index[rand_index]] - s
            w += alpha * error * data
            del (data_index[rand_index])
    return w

w = grad_ascent(train_data, train_lab)
predict_lab = np.round(sigmoid(np.dot(test_data, w)))
fault_cnt = 0
for i in range(np.shape(test_data)[0]):
    if not predict_lab[i] == test_lab[i]:
        fault_cnt += 1
print("accuracy: ", 1 - fault_cnt / np.shape(test_data)[0])


accuracy:  0.73
